# 5. LazyClassifierを使ってみる。

In [ ]:
from IPython.display import clear_output
!pip3 install -U lazypredict
!pip3 install -U pandas #Upgrading pandas

clear_output()

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots



from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, train_test_split


from lightgbm import LGBMClassifier
import lazypredict
from lazypredict.Supervised import LazyClassifier

import time
import warnings
warnings.filterwarnings('ignore')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

train = pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
test = pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')

In [ ]:
train.head()

In [ ]:
import random, math

train_Cabin, test_Cabin = train['Cabin'].values, test['Cabin'].values
unique_CabinDeck = ['B', 'F', 'A', 'G', 'E', 'D', 'C', 'T']
unique_CabinSide = ['P', 'S']

train_Cabin_new = []
for train_c in train_Cabin:
    randCabinDeck = random.choice(unique_CabinDeck)
    randCabinNum = random.randint(0,1894)
    randCabinSide = random.choice(unique_CabinSide)
    if pd.isnull(train_c):
        train_c_new = str(randCabinDeck) + '/' + str(randCabinNum) + '/' + str(randCabinSide)
        train_Cabin_new.append(train_c_new)
    else:
        train_c_new = train_c
        train_Cabin_new.append(train_c_new)

test_Cabin_new = []
for test_c in test_Cabin:
    randCabinDeck = random.choice(unique_CabinDeck)
    randCabinNum = random.randint(0,1894)
    randCabinSide = random.choice(unique_CabinSide)
    if pd.isnull(test_c):
        test_c_new = str(randCabinDeck) + '/' + str(randCabinNum) + '/' + str(randCabinSide)
        test_Cabin_new.append(test_c_new)
    else:
        test_c_new = test_c
        test_Cabin_new.append(test_c_new)
        
train['Cabin_new'], test['Cabin_new'] = train_Cabin_new, test_Cabin_new
del train['Cabin'], test['Cabin']

PIds_train,PIds_test = train['PassengerId'].values, test['PassengerId'].values
Cabins_train, Cabins_test= train['Cabin_new'].values, test['Cabin_new'].values

group_nums_train  = []
num_within_groups_train = []
cabin_decks_train = []
cabin_nums_train = []
cabin_sides_train = []
group_nums_test  = []
num_within_groups_test = []
cabin_decks_test = []
cabin_nums_test = []
cabin_sides_test = []

for PId_train in PIds_train:
    group_num_train,num_within_group_train = PId_train.split('_')
    group_nums_train.append(group_num_train)
    num_within_groups_train.append(num_within_group_train)

for Cabin_train in Cabins_train:
    cabin_deck_train,cabin_num_train,cabin_side_train = Cabin_train.split('/')
    cabin_decks_train.append(cabin_deck_train)
    cabin_nums_train.append(cabin_num_train)
    cabin_sides_train.append(cabin_side_train)

for PId_test in PIds_test:
    group_num_test,num_within_group_test = PId_test.split('_')
    group_nums_test.append(group_num_test)
    num_within_groups_test.append(num_within_group_test)

for Cabin_test in Cabins_test:
    cabin_deck_test,cabin_num_test,cabin_side_test = Cabin_test.split('/')
    cabin_decks_test.append(cabin_deck_test)
    cabin_nums_test.append(cabin_num_test)
    cabin_sides_test.append(cabin_side_test)
    
train['GroupNum'] = group_nums_train
train['NumWithinGroup'] = num_within_groups_train
train['CabinDeck'] = cabin_decks_train
train['CabinNum'] = cabin_nums_train
train['CabinSide'] = cabin_sides_train
test['GroupNum'] = group_nums_test
test['NumWithinGroup'] = num_within_groups_test
test['CabinDeck'] = cabin_decks_test
test['CabinNum'] = cabin_nums_test
test['CabinSide'] = cabin_sides_test
del train['Cabin_new'],test['Cabin_new']

In [ ]:
Decks_train, Decks_test = train['CabinDeck'].values, test['CabinDeck'].values
CabinDeck_new_train = []
CabinDeck_new_test = []

for Deck_train in Decks_train:
    if Deck_train == 'D' or Deck_train == 'E' or Deck_train == 'F' or Deck_train == 'T':
        Deck_layer_train = 'LowRisk'
    elif Deck_train == 'G' or Deck_train == 'A':
        Deck_layer_train = 'MiddleRisk'
    elif Deck_train == 'B' or Deck_train == 'C':
        Deck_layer_train = 'HighRisk'
    CabinDeck_new_train.append(Deck_layer_train)

for Deck_test in Decks_test:
    if Deck_test == 'D' or Deck_test == 'E' or Deck_test == 'F' or Deck_test == 'T':
        Deck_layer_test = 'LowRisk'
    elif Deck_test == 'G' or Deck_test == 'A':
        Deck_layer_test = 'MiddleRisk'
    elif Deck_test == 'B' or Deck_test == 'C':
        Deck_layer_test = 'HighRisk'
    CabinDeck_new_test.append(Deck_layer_test)
    
train['CabinDeck_new'], test['CabinDeck_new'] = CabinDeck_new_train, CabinDeck_new_test
del train['CabinDeck'], test['CabinDeck']


train = pd.get_dummies(train, columns=['CabinDeck_new'])
test = pd.get_dummies(test, columns=['CabinDeck_new'])

CabinSide_S_train, CabinSide_S_test = train['CabinSide'].replace({'P':0,'S':1}), test['CabinSide'].replace({'P':0,'S':1})
train['isCabinSide_S'], test['isCabinSide_S'] = CabinSide_S_train, CabinSide_S_test

FamilySize_train, FamilySize_test = train['GroupNum'].value_counts(), test['GroupNum'].value_counts()
FamilySize_dict_train, FamilySize_dict_test = FamilySize_train.to_dict(), FamilySize_test.to_dict()
FamilySize_new_train, FamilySize_new_test = train['GroupNum'].replace(FamilySize_dict_train), test['GroupNum'].replace(FamilySize_dict_test)
train['FamilySize'], test['FamilySize'] = FamilySize_new_train, FamilySize_new_test
del train['GroupNum'], train['NumWithinGroup'], train['CabinSide'], test['GroupNum'], test['NumWithinGroup'], test['CabinSide']



In [ ]:
train_homeplanet, test_homeplanet = train['HomePlanet'].values, test['HomePlanet'].values
unique_homeplanet = ['Europa', 'Earth', 'Mars']

train_homeplanet_new = []
for train_h in train_homeplanet:
    randHomePlanet = random.choice(unique_homeplanet)
    if pd.isnull(train_h):
        train_h_new = randHomePlanet
        train_homeplanet_new.append(train_h_new)
    else:
        train_h_new = train_h
        train_homeplanet_new.append(train_h_new)

test_homeplanet_new = []
for test_h in test_homeplanet:
    randHomePlanet = random.choice(unique_homeplanet)
    if pd.isnull(test_h):
        test_h_new = randHomePlanet
        test_homeplanet_new.append(test_h_new)
    else:
        test_h_new = test_h
        test_homeplanet_new.append(test_h_new)
        
train['HomePlanet_new'],test['HomePlanet_new'] = train_homeplanet_new, test_homeplanet_new
del train['HomePlanet'],test['HomePlanet']

# CryoSleep:ランダム補完


train_CryoSleep, test_CryoSleep = train['CryoSleep'].values, test['CryoSleep'].values
unique_CryoSleep = [True, False]

train_CryoSleep_new = []
for train_cryo in train_CryoSleep:
    randCryoSleep = random.choice(unique_CryoSleep)
    if pd.isnull(train_cryo):
        train_cryo_new = randCryoSleep
        train_CryoSleep_new.append(train_cryo_new)
    else:
        train_cryo_new = train_cryo
        train_CryoSleep_new.append(train_cryo_new)
        
test_CryoSleep_new = []
for test_cryo in test_CryoSleep:
    randCryoSleep = random.choice(unique_CryoSleep)
    if pd.isnull(test_cryo):
        test_cryo_new = randCryoSleep
        test_CryoSleep_new.append(test_cryo_new)
    else:
        test_cryo_new = test_cryo
        test_CryoSleep_new.append(test_cryo_new)        

train['CryoSleep_new'], test['CryoSleep_new'] = train_CryoSleep_new, test_CryoSleep_new
del train['CryoSleep'], test['CryoSleep']

# Destination:最頻値補完

train_Des, test_Des = train['Destination'].fillna('TRAPPIST-1e'), test['Destination'].fillna('TRAPPIST-1e')
train['Destination_new'], test['Destination_new'] = train_Des, test_Des
del train['Destination'], test['Destination']

# Age:最頻値補完

Age_train, Age_test = train['Age'].fillna(train['Age'].median()), test['Age'].fillna(train['Age'].median())
train['Age_new'] = Age_train
test['Age_new'] = Age_test
del train['Age'], test['Age']

# VIP:最頻値補完
train_Vip, test_Vip = train['VIP'].fillna(False), test['VIP'].fillna(False)
train['VIP_new'], test['VIP_new'] = train_Vip, test_Vip
del train['VIP'], test['VIP']

#ワンホット
train = pd.get_dummies(train, columns=['HomePlanet_new','Destination_new'])
test = pd.get_dummies(test, columns=['HomePlanet_new','Destination_new'])
IsCryoSleep_train, IsCryoSleep_test = train['CryoSleep_new'].replace({False:0,True:1}), test['CryoSleep_new'].replace({False:0,True:1})
train['IsCryoSleep'], test['IsCryoSleep'] = IsCryoSleep_train, IsCryoSleep_test
IsNotVIP_train, IsNotVIP_test = train['VIP_new'].replace({False:1,True:0}), test['VIP_new'].replace({False:1,True:0})
train['IsNotVIP'], test['IsNotVIP'] = IsNotVIP_train, IsNotVIP_test

del train['CryoSleep_new'],train['VIP_new'],train['Name'],test['CryoSleep_new'],test['VIP_new'],test['Name']
train['CabinNum'], test['CabinNum'] = train['CabinNum'].astype('int64'), test['CabinNum'].astype('int64')

In [ ]:
RSs_train, RSs_test = train['RoomService'].fillna(0).values, test['RoomService'].fillna(0).values
FCs_train, FCs_test = train['FoodCourt'].fillna(0).values, test['FoodCourt'].fillna(0).values
SMs_train, SMs_test = train['ShoppingMall'].fillna(0).values, test['ShoppingMall'].fillna(0).values
SPs_train, SPs_test = train['Spa'].fillna(0).values, test['Spa'].fillna(0).values
VRs_train, VRs_test = train['VRDeck'].fillna(0).values, test['VRDeck'].fillna(0).values
PaidService_train = RSs_train + FCs_train + SMs_train + SPs_train + VRs_train
PaidService_test = RSs_test + FCs_test + SMs_test + SPs_test + VRs_test

train['PaidService'], test['PaidService'] = PaidService_train, PaidService_test
del train['RoomService'],train['FoodCourt'],train['ShoppingMall'],train['Spa'],train['VRDeck'],test['RoomService'],test['FoodCourt'],test['ShoppingMall'],test['Spa'],test['VRDeck']

In [ ]:
from sklearn.preprocessing import StandardScaler

X = train[['CabinNum','CabinDeck_new_HighRisk','CabinDeck_new_LowRisk','CabinDeck_new_MiddleRisk', 'isCabinSide_S', 'FamilySize', 'Age_new', 'HomePlanet_new_Earth', 'HomePlanet_new_Europa', 'HomePlanet_new_Mars', 'Destination_new_55 Cancri e', 'Destination_new_PSO J318.5-22', 'Destination_new_TRAPPIST-1e', 'IsCryoSleep', 'IsNotVIP', 'PaidService']]
y = train[['Transported']]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, shuffle=True, random_state=0)

scaler = StandardScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_val_scaled = scaler.transform(X_val)

In [ ]:
lc = LazyClassifier(verbose=0,
                    ignore_warnings=True,
                    custom_metric=None,
                    predictions=False,
                    random_state=12,
                    classifiers='all')

models, predictions = lc.fit(X_train , X_val , y_train , y_val)
clear_output()

In [ ]:
models[:15]

In [ ]:
line = px.line(data_frame= models ,y =["Accuracy"] , markers = True)
line.update_xaxes(title="Model",
              rangeslider_visible = False)
line.update_yaxes(title = "Accuracy")
line.update_traces(line_color="red")
line.update_layout(showlegend = True,
    title = {
        'text': 'Accuracy vs Model',
        'y':0.94,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})

line.show()

In [ ]:
line = px.line(data_frame= models ,y =["ROC AUC" , "F1 Score"] , markers = True)
line.update_xaxes(title="Model",
              rangeslider_visible = False)
line.update_yaxes(title = "ROC AUC SCORE")
line.update_layout(showlegend = True,
    title = {
        'text': 'ROC AUC and F1 Score vs Model',
        'y':0.94,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})

line.show()

In [ ]:
line = px.line(data_frame= models ,y =["Time Taken"] , markers = True)
line.update_xaxes(title="Model",
              rangeslider_visible = False)
line.update_yaxes(title = "Time(s)")
line.update_traces(line_color="purple")
line.update_layout(showlegend = True,
    title = {
        'text': 'TIME TAKEN vs Model',
        'y':0.94,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})

line.show()

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV

gbm = lgb.LGBMClassifier(objective='binary')

# params = {
#     'num_leaves': [3, 4, 5, 6, 7, 8, 9, 10],
#     'reg_alpha': [0, 1, 2, 3, 4, 5,10, 100],
#     'reg_lambda': [10, 15, 18, 20, 21, 22, 23, 25, 27, 29]
# }

# grid_search = GridSearchCV(gbm, param_grid=params, cv=5)
# grid_search.fit(X_train_scaled, y_train.values.ravel())

# print(grid_search.best_score_)
# print(grid_search.best_params_)

0.7557929334428923
{'num_leaves': 9, 'reg_alpha': 10, 'reg_lambda': 25}

In [ ]:
# params2 = {
#     'num_leaves': [8.4, 8.7, 9, 9.3, 9.6],
#     'reg_alpha': [7,10,15,20,50],
#     'reg_lambda': [24, 25, 26]
# }

# grid_search2 = GridSearchCV(gbm, param_grid=params2, cv=5)
# grid_search2.fit(X_train_scaled, y_train.values.ravel())

# print(grid_search2.best_score_)
# print(grid_search2.best_params_)

0.7557929334428923
{'num_leaves': 9, 'reg_alpha': 10, 'reg_lambda': 25}

In [ ]:
lgb_best = lgb.LGBMClassifier(num_leaves=9, reg_alpha=10, reg_lambda=25)
lgb_best.fit(X_train_scaled,y_train)
lgb_best.score(X_val_scaled,y_val)

In [ ]:
X_for_submit = test[['CabinNum','CabinDeck_new_HighRisk','CabinDeck_new_LowRisk','CabinDeck_new_MiddleRisk','isCabinSide_S','FamilySize','Age_new', 'HomePlanet_new_Earth', 'HomePlanet_new_Europa', 'HomePlanet_new_Mars', 'Destination_new_55 Cancri e', 'Destination_new_PSO J318.5-22', 'Destination_new_TRAPPIST-1e', 'IsCryoSleep', 'IsNotVIP', 'PaidService']]
submit = test[['PassengerId']]

X_for_submit_scaled = scaler.transform(X_for_submit)
submit['Transported'] = lgb_best.predict(X_for_submit_scaled)
submit.to_csv('submit_lc.csv', index=False)